In [212]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [224]:
nba_url = 'https://www.spotrac.com/nfl/draft/'
year = '2017'

In [225]:
r = requests.get((nba_url + year))

In [226]:
soup = BeautifulSoup(r.text, 'html.parser')

In [227]:
table_headers = soup.find_all('th')[0:10]
column_names = []
for x in table_headers:
    column_names.append(x.text)
column_names

['Pick',
 'Team',
 'Player',
 'Pos',
 'Age',
 'College',
 'Yrs',
 'Total Value',
 'Signing Bonus',
 '2017 Cap']

In [228]:

df = pd.DataFrame(columns = column_names, index = [i for i in range(256)])

In [229]:
table_data = soup.find_all('td')
for i in range(10):
    print(table_data[i].text)
len(table_data)

1  
CLE 
 Myles Garrett 
DE
21
Texas A&M
4
$30,412,255
$20,258,004
$5,529,501


2530

In [233]:
cnt = -1
for i in range(len(table_data)):
    if(i % 10 == 0):
        cnt += 1
#     print(i)
    temp = table_data[i]
    temp_text = temp.text
    df.iat[cnt, i%10] = temp_text

In [234]:
df

,Pick,Team,Player,Pos,Age,College,Yrs,Total Value,Signing Bonus,2017 Cap
0,1,CLE,Myles Garrett,DE,21,Texas A&M,4,"$30,412,255","$20,258,004","$5,529,501"
1,2,CHI from SF,Mitchell Trubisky,QB,22,North Carolina,4,"$29,032,424","$19,254,490","$5,278,622"
2,3,SF from CHI,Solomon Thomas,DE,21,Stanford,4,"$28,154,366","$18,615,903","$5,118,975"
3,4,JAC,Leonard Fournette,RB,22,Louisiana State,4,"$27,150,842","$17,886,067","$4,936,516"
4,5,TEN from LA,Corey Davis,WR,22,Western Michigan,4,"$25,394,688","$16,608,864","$4,617,216"
5,6,NYJ,Jamal Adams,SS,21,Louisiana State,4,"$22,258,738","$14,328,154","$4,047,038"
6,7,LAC,Mike Williams,WR,22,Clemson,4,"$19,749,928","$12,503,584","$3,590,896"
7,8,CAR,Christian McCaffrey,RB,20,Stanford,4,"$17,241,304","$10,679,128","$3,134,782"
8,9,CIN,John Ross,WR,22,Washington,4,"$17,115,863","$10,587,900","$3,084,444"
9,10,KC from BUF,Patrick Mahomes,QB,21,Texas Tech,4,"$16,425,786","$10,086,026","$2,986,506"


In [153]:
df['year'] = year

In [155]:
df.to_csv(f'nba_draft_player_salaries_{year}.csv',index=False)

In [182]:
#sport argument accepts 'nba', 'nfl', 'mlb' as acceptable arguments
#year is the year that is requested
#rows_count is the number of rows that are to be pulled from the requested site
def scrape_salaries_nba(year):
    
    r = requests.get(f'https://www.spotrac.com/nba/draft/' + year)
    soup = BeautifulSoup(r.text, 'html.parser')
    table_data = soup.find_all('td')
    
    columns = ['pick', 'team', 'player', 'pos', 'height', 'weight', 'country', 'college', 'years', 'dollars', 'year cap hit']
    df = pd.DataFrame(columns = columns, index = [i for i in range(60)])
    
    cnt = -1
    for i in range(len(table_data)):
        if(i % 11 == 0):
            cnt += 1
        df.iat[cnt, i%11] = table_data[i].text
    
    df['year'] = year
    df.to_csv(f'nba_draft_player_salaries_{year}.csv',index=False)

In [241]:
r.status_code

200

In [242]:
def scrape_salaries(sport, year, rows):
    
    #requests site specific to sport and selects all relevant data
    r = requests.get(f'https://www.spotrac.com/{sport}/draft/{year}/')
    
    if r.status_code == 200:

        soup = BeautifulSoup(r.text, 'html.parser')
        table_data = soup.find_all('td')
        table_headers = soup.find_all('th')[0:10]

        #gets column names from table_headers on requested page
        column_names = []
        for x in table_headers:
            column_names.append(x.text)

        #created dataframe to store data
        df = pd.DataFrame(columns = column_names, index = [i for i in range(rows)])
        #loops through requested data and appends it to table in very smart manner
        cnt = -1
        for i in range(len(table_data)):
            if(i % len(table_headers) == 0):
                cnt += 1
            df.iat[cnt, i % len(table_headers)] = table_data[i].text
        #attaches year to dataframe and exports it as .csv
        df['year'] = year
        df.to_csv(f'{sport}_draft_player_salaries_{year}.csv',index=False)
    else:
        return 'Request not successful, Code: ' + r.status_code

In [254]:
scrape_salaries('nfl', '2005', 256)